In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 28 21:46:23 2016

@author: stephen
"""

from __future__ import print_function

from tensorflow import keras
import numpy as np
import pandas as pd

np.random.seed(813306)


def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y

# epochs = 5000
epochs = 10

flist = ["../../datasets/MixedShapesSmallTrain/MixedShapesSmallTrain"]
for each in flist:
    fname = each
    x_train, y_train = readucr(fname + "_TRAIN.txt")
    x_test, y_test = readucr(fname + "_TEST.txt")
    nb_classes = len(np.unique(y_test))
    y_train = (
        (y_train - y_train.min()) / (y_train.max() - y_train.min()) * (nb_classes - 1)
    )
    y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min()) * (nb_classes - 1)
    batch_size = int(min(x_train.shape[0] / 10, 16))

    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)

    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / (x_train_std)

    # x_test_min = np.min(x_test, axis = 1, keepdims=1)
    # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean) / (x_train_std)

    # x_train = x_train.reshape(x_train.shape + (1,))
    # x_test = x_test.reshape(x_test.shape + (1,))

    x = keras.layers.Input(x_train.shape[1:])
    y = keras.layers.Dropout(0.1)(x)
    y = keras.layers.Dense(500, activation="relu")(y)
    y = keras.layers.Dropout(0.2)(y)
    y = keras.layers.Dense(500, activation="relu")(y)
    y = keras.layers.Dropout(0.2)(y)
    y = keras.layers.Dense(500, activation="relu")(y)
    y = keras.layers.Dropout(0.3)(y)
    out = keras.layers.Dense(nb_classes, activation="softmax")(y)

    model = keras.models.Model(inputs=x, outputs=out)

    optimizer = keras.optimizers.Adadelta()
    model.compile(
        loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="loss", factor=0.5, patience=200, min_lr=0.1
    )

    hist = model.fit(
        x_train,
        Y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, Y_test),
        callbacks=[reduce_lr],
    )


Epoch 1/10
10/10 [==============================] - 3s 209ms/step - loss: 1.9000 - accuracy: 0.1800 - val_loss: 1.6944 - val_accuracy: 0.2553 - lr: 0.0010
Epoch 2/10
10/10 [==============================] - 1s 130ms/step - loss: 1.9384 - accuracy: 0.1700 - val_loss: 1.6823 - val_accuracy: 0.2553 - lr: 0.0010
Epoch 3/10
10/10 [==============================] - 1s 151ms/step - loss: 1.7857 - accuracy: 0.2300 - val_loss: 1.6712 - val_accuracy: 0.2565 - lr: 0.0010
Epoch 4/10
10/10 [==============================] - 1s 150ms/step - loss: 1.7970 - accuracy: 0.2000 - val_loss: 1.6602 - val_accuracy: 0.2590 - lr: 0.0010
Epoch 5/10
10/10 [==============================] - 1s 149ms/step - loss: 1.9760 - accuracy: 0.1700 - val_loss: 1.6487 - val_accuracy: 0.2598 - lr: 0.0010
Epoch 6/10
10/10 [==============================] - 1s 151ms/step - loss: 1.8334 - accuracy: 0.1800 - val_loss: 1.6381 - val_accuracy: 0.2623 - lr: 0.0010
Epoch 7/10
10/10 [==============================] - 1s 156ms/step - lo